In [47]:
import pandas as pd
import numpy as np

In [48]:
ds_train = pd.read_csv('ds/train.csv')
ds_songs = pd.read_csv('ds/songs.csv')
ds_songs_extra_info = pd.read_csv('ds/song_extra_info.csv')
ds_members = pd.read_csv('ds/members.csv')

In [49]:
ds_train.head(1)

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1


In [50]:
ds_songs.head(1)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0


Процент песен без жанров

In [51]:
print(ds_songs['genre_ids'].isna().sum() / ds_songs.shape[0])

0.04098557692307692


In [52]:
genres_for_song = ds_songs['genre_ids'].apply(lambda x: [int(v) for v in str(x).split('|') if v != 'nan'])

In [53]:
genres = dict()
for genre_list in genres_for_song:
    for g in genre_list:
        if g not in genres:
            genres[g] = 0
        genres[g] += 1

In [54]:
sorted_genres = {k: v for k, v in reversed(sorted(genres.items(), key=lambda item: item[1]))}
sorted_genres['0'] = -1

In [55]:
def get_most_popular(g_list):
  res = '0'
  for g in g_list:
    if sorted_genres[res] < sorted_genres[g]:
      res = g
  return res

In [56]:
most_pop = []
for genre_list in genres_for_song:
  most_pop.append(get_most_popular(genre_list))

In [57]:
ds_songs['genre_ids'] = most_pop

In [58]:
ds_songs.head(10)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0
5,kKJ2JNU5h8rphyW21ovC+RZU+yEHPM+3w85J37p7vEQ=,235520,843,貴族精選,Joe Hisaishi,Hayao Miyazaki,17.0
6,N9vbanw7BSMoUgdfJlgX1aZPE1XZg8OS1wf88AQEcMc=,226220,458,伍佰 & China Blue,Jonathan Lee,NaN,3.0
7,GsCpr618xfveHYJdo+E5SybrpR906tsjLMeKyrCNw8s=,276793,465,光良 (Michael Wong),光良,彭資閔,3.0
8,oTi7oINPX+rxoGp+3O6llSltQTl80jDqHoULfRoLcG4=,228623,465,林俊傑 (JJ Lin),JJ Lin,Wu Qing Feng,3.0
9,btcG03OHY3GNKWccPP0auvtSbhxog/kllIIOx5grE/k=,232629,352,Kodaline,Stephen Garrigan| Mark Prendergast| Vincent Ma...,Stephen Garrigan| Mark Prendergast| Vincent Ma...,52.0


## Some header

In [59]:
def leave_first(column):
    items = ds_songs[column].apply(lambda x: [v for v in str(x).split('|')])
    res = [v[0] for v in items]
    ds_songs[column] = res

In [60]:
leave_first('artist_name')
leave_first('composer')
leave_first('lyricist')

In [61]:
ds_songs.head(3)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,nan,nan,31.0


## SomeHeader

In [62]:
ds_train = ds_train.merge(ds_members,on='msno',how='left')
ds_train = ds_train.merge(ds_songs,on='song_id',how='left')
ds_train = ds_train.fillna(0)

In [63]:
pd.set_option('display.max_columns', None)
ds_train.head(3)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,bd,gender,registered_via,registration_init_time,expiration_date,song_length,genre_ids,artist_name,composer,lyricist,language
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,1,0,0,7,20120102,20171005,206471.0,359,Bastille,Dan Smith,nan,52.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,13,24,female,9,20110525,20170911,284584.0,1259,Various Artists,nan,nan,52.0
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,13,24,female,9,20110525,20170911,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,nan,52.0


In [65]:
ds_train.drop(columns=['registration_init_time','expiration_date','song_length'], inplace=True)

## SomeHeader

In [64]:
ds_train['language'] = ds_train['language'].astype(int).astype('category')
ds_train.head(3)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,city,bd,gender,registered_via,registration_init_time,expiration_date,song_length,genre_ids,artist_name,composer,lyricist,language
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,1,0,0,7,20120102,20171005,206471.0,359,Bastille,Dan Smith,nan,52
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,13,24,female,9,20110525,20170911,284584.0,1259,Various Artists,nan,nan,52
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1,13,24,female,9,20110525,20170911,225396.0,1259,Nas,N. Jones、W. Adams、J. Lordan、D. Ingle,nan,52


In [19]:
X_train=ds_train.sample(frac=0.8)
X_test=ds_train.drop(X_train.index)

In [20]:
y_train = X_train['target']
y_test = X_test['target']
X_train = X_train.drop(columns=['target'])
X_test = X_test.drop(columns=['target'])

In [21]:
from sklearn.metrics import roc_auc_score

In [22]:
from catboost import CatBoostClassifier

In [23]:
X_train.head(3)

,msno,song_id,source_system_tab,source_screen_name,source_type,city,bd,gender,registered_via,registration_init_time,expiration_date,song_length,genre_ids,artist_name,composer,lyricist,language
1886047,Jvn/e4wagLZK8hu4nTF+NcYdXUDnuhFcn7PEjx0ZWjA=,4ZbS9i8tryjCVE7gLjG5oNC5aAasntVUHzIs90LjXxM=,discover,Online playlist more,online-playlist,7,17,female,9,20110224,20171017,222249.0,139,Chris Brown,Chris Brown; Jean Baptiste; Michael McHenry; R...,nan,52
107864,6rZ/0zGskDiqdR4CSx7IzmLiO7QskqgRxiReqwoLmqQ=,vSdM6LZCFclerFHXPav5sX6zLEZrY0dxkOeAM8Rw3wI=,my library,Local playlist more,local-playlist,22,28,male,3,20130621,20171027,266971.0,465,莊心妍,潘攀,潘攀,3
6459689,nbytZ3+EdbMM3Rk80k4v4g4t4HPbtUAmAIGXYw31mWc=,IBXJqj7sjP+Dv94TRn/qrUaLei78KLV+qabZz4Efvs8=,my library,Local playlist more,local-playlist,15,37,male,3,20120725,20171005,201456.0,2122,Billie Holiday,Sinatra,nan,52


In [24]:
X_train.shape
# X_train = X_train.drop(columns=['registration_init_time','expiration_date','song_length'])
# X_test = X_test.drop(columns=['registration_init_time','expiration_date','song_length'])

In [25]:
X_train.shape

(5901934, 14)

In [26]:
cat_features = [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13]

In [35]:
clf = CatBoostClassifier(iterations=200, verbose=20, cat_features=cat_features)

In [36]:
clf.fit(X_train, y_train)

Learning rate set to 0.5
0:	learn: 0.6012561	total: 11.5s	remaining: 38m 17s
20:	learn: 0.5139671	total: 7m 15s	remaining: 1h 1m 55s
40:	learn: 0.5084448	total: 12m 28s	remaining: 48m 21s
60:	learn: 0.5054544	total: 18m 1s	remaining: 41m 3s
80:	learn: 0.5035891	total: 23m 29s	remaining: 34m 30s
100:	learn: 0.5024082	total: 28m 14s	remaining: 27m 40s
120:	learn: 0.5016197	total: 33m 12s	remaining: 21m 41s
140:	learn: 0.5009643	total: 37m 58s	remaining: 15m 53s
160:	learn: 0.5003874	total: 43m 15s	remaining: 10m 28s
180:	learn: 0.4998241	total: 48m 35s	remaining: 5m 6s
199:	learn: 0.4994610	total: 52m 53s	remaining: 0us


In [37]:
prediction = clf.predict(X_test)

In [38]:
print(roc_auc_score(y_test, prediction))

0.7696584266039791


In [66]:
from sklearn.model_selection import KFold

In [67]:
kf = KFold(n_splits=5)

In [68]:
ds_train = ds_train.sample(frac=1)

In [69]:
y = ds_train['target']
ds_train.drop(columns=['target'], inplace=True)

In [70]:
y = y.values
ds_train = ds_train.values

In [71]:
auc = 0

In [72]:
for train_index, test_index in kf.split(ds_train):
    X_train, X_test = ds_train[train_index], ds_train[test_index]
    y_train, y_test = y[train_index], y[test_index]
    clf = CatBoostClassifier(iterations=200, verbose=50, cat_features=cat_features)
    clf.fit(X_train, y_train)
    prediction = clf.predict(X_test)
    auc += roc_auc_score(y_test, prediction)

Learning rate set to 0.5
0:	learn: 0.6012035	total: 12.4s	remaining: 41m 2s
50:	learn: 0.5065397	total: 12m 31s	remaining: 36m 35s
100:	learn: 0.5021604	total: 24m 16s	remaining: 23m 48s
150:	learn: 0.5002272	total: 35m 58s	remaining: 11m 40s
199:	learn: 0.4990657	total: 47m 23s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.6012531	total: 12.4s	remaining: 41m 7s
50:	learn: 0.5064885	total: 11m 51s	remaining: 34m 38s
100:	learn: 0.5025385	total: 23m 18s	remaining: 22m 51s
150:	learn: 0.5004984	total: 34m 56s	remaining: 11m 20s
199:	learn: 0.4992940	total: 46m 2s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.6013114	total: 11.7s	remaining: 38m 55s
50:	learn: 0.5065345	total: 11m 43s	remaining: 34m 16s
100:	learn: 0.5026200	total: 23m 38s	remaining: 23m 10s
150:	learn: 0.5005150	total: 35m 15s	remaining: 11m 26s
199:	learn: 0.4993737	total: 46m 59s	remaining: 0us
Learning rate set to 0.5
0:	learn: 0.6012906	total: 11.5s	remaining: 38m 11s
50:	learn: 0.5065395	total: 11m 30s	re

In [74]:
auc / 5

0.7693701048448058